# Diffusion embedding of functional connectivity matrix

In [ ]:
import h5py, sys, io, numpy as np
#from mapalign import embed, dist

#embedding, result = embed.compute_diffusion_map(L, alpha=0.5, n_components=5, diffusion_time=0, skip_checks=False, overwrite=True)


In [ ]:
import scipy.io
emb = scipy.io.loadmat('../data/hcp.embed.all.313.mat')

## Visualization on the cortical surface

In [ ]:
import nibabel as nib 

surfmL = nib.freesurfer.read_geometry('gradient_data/templates/Q1-Q6_R440.L.midthickness.32k_fs_LR.surf')
surfiL = nib.freesurfer.read_geometry('gradient_data/templates/Q1-Q6_R440.L.very_inflated.32k_fs_LR.surf')
surfL = []
surfL.append(np.array(surfmL[0]*0.3 + surfiL[0]*0.7))
surfL.append(surfmL[1])

surfmR = nib.freesurfer.read_geometry('gradient_data/templates/Q1-Q6_R440.R.midthickness.32k_fs_LR.surf')
surfiR = nib.freesurfer.read_geometry('gradient_data/templates/Q1-Q6_R440.R.very_inflated.32k_fs_LR.surf')
surfR = []
surfR.append(np.array(surfmR[0]*0.3 + surfiR[0]*0.7))
surfR.append(surfmR[1])

sulcL = -1 * nib.load('gradient_data/templates/Q1-Q6_R440.sulc.32k_fs_LR.dscalar.nii').data[0:32492]
sulcR = -1 * nib.load('gradient_data/templates/Q1-Q6_R440.sulc.32k_fs_LR.dscalar.nii').data[32492:32492*2]

res = nib.load('gradient_data/templates/hcp.embed.all.179.lh.dscalar.nii').data
cortL = np.squeeze(np.array(np.where(res != 0)[0], dtype=np.int32))
res = nib.load('gradient_data/templates/hcp.embed.all.179.rh.dscalar.nii').data
cortR = np.squeeze(np.array(np.where(res != 0)[0], dtype=np.int32))
del res

In [ ]:
% matplotlib inline
import sys
sys.path.append('gradient_data/src/')
from viz import *

In [ ]:
# Scatterplot of first two dimensions

# Macaque monkey connectivity data gradients

# Geodesic distance from gradient peaks

In [ ]:
# scatterplot

# Plotting on network templates

In [ ]:
import nibabel as nib
import nibabel.gifti
from matplotlib.colors import ListedColormap
import nilearn
from nilearn import datasets, plotting
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## Yeo7 networks 

In [ ]:
yeo7L = nib.gifti.read('gradient_data/templates/hcp.RSN.L.gii').darrays[0].data
yeo7R = nib.gifti.read('gradient_data/templates/hcp.RSN.R.gii').darrays[0].data
yeo17L = nib.gifti.read('gradient_data/templates/hcp.RSN.L.gii').darrays[1].data
yeo17R = nib.gifti.read('gradient_data/templates/hcp.RSN.R.gii').darrays[1].data

In [ ]:
yeo7 = np.concatenate((yeo7L[cortL], yeo7L[cortR]))
uniqueY = np.array(np.unique(yeo7), dtype=np.int32)[1::]
sz = len(uniqueY)
rsns={}
for c in range(sz):
    rsns['k'+str(int(c)+1)]=[]

r = []     
for x in range(sz):
    g = emb['embedding'][np.where(yeo7 == uniqueY[x]),0]#emb_rot[np.where(yeo7 == uniqueY[x]),0]#
    for y in g[0]:
        if y != 0:
            rsns['k'+str(int(x)+1)].append(float(y))
    r.append(np.mean(rsns['k'+str(int(x)+1)]))

ord = np.argsort(r)

plot_list=[]
for c in range(sz):    
    plot_list+=[np.squeeze(rsns['k'+str(ord[c]+1)])]

names=[]
for c in range(sz):
    names.append(str(int(ord[c])+1))

convertYeo7 = [2,5,6,0,4,1,3] 
cols7 = ((255,255,255),
         (120,  18, 134),
         (70 ,130, 180),
         ( 0 ,118 , 14),
         (196 , 58 ,250),
         (220 ,248 ,164),
         (230, 148,  34),
         (205 , 62,  78 ))
cols = cols7
cols = np.asarray(cols, dtype=float)/255
yeoCols = ListedColormap(cols,name='colormapYeo')

colors1 = list()
for i in xrange(0,len(ord)):
    colors1.append(tuple(cols[convertYeo7[ord[i]] + 1]))

In [ ]:
height=10
sns.set_context('notebook', font_scale=1.8)
sns.axes_style("white")
sns.set_palette(colors1)

fig = plt.figure(figsize=(20,height))
ax = sns.boxplot(data=plot_list, saturation=0.5, orient='h', flierprops=dict(marker='+', markersize=2, 
                                                                             color="gray", markeredgecolor="gray"))
ax.set_yticklabels('')
sns.axlabel('Gradient Values','', fontsize=22)
plt.savefig('gradient_data/figures/yeo_dist_plot.png')
plt.close(fig)

fig, axarr = plt.subplots(nrows=sz, ncols=1, figsize=(2*(20/sz),height))
data = datasets.fetch_atlas_yeo_2011(data_dir='./')
yeoV7 = nibabel.load(data.thick_7)
y = 0
for x in range(sz):
    yeoV7_1 = nibabel.Nifti1Image((yeoV7.get_data() == convertYeo7[ord[x]] + 1).astype(int), yeoV7.get_affine())     
    yeoCols = ListedColormap(cols[[0,convertYeo7[ord[x]] + 1]],name='colormapYeo')
    plotting.plot_glass_brain(yeoV7_1, axes=axarr[x], annotate=False, alpha=0.3, cmap=yeoCols, threshold=0.1, 
                          output_file='gradient_data/figures/yeo_brains.png')
    y += 1

from PIL import Image
i1 = Image.open('gradient_data/figures/yeo_dist_plot.png')
i2 = Image.open('gradient_data/figures/yeo_brains.png')
result = Image.new("RGBA", (np.shape(i1)[1]+50, np.shape(i1)[0]))
result.paste(i1, (120, 0))
result.paste(i2, (0, 0))
result.save('gradient_data/figures/Yeo7_Fig.png')

In [ ]:
from IPython.display import Image as im
im(filename='gradient_data/figures/Yeo7_Fig.png') 

In [ ]:
y = yeo7L.copy()
for n, v in enumerate(uniqueY):
    y[yeo7L == v] = int(convertYeo7[n]) - 3
y[yeo7L == 37] = - 4
c = np.where([yeo17L != 37])[1]
f = plot_surf_stat_map(surfL[0], surfL[1], bg_map=sulcL, mask=c, bg_on_stat=True, stat_map=y, azim=0, alpha=1, cmap=yeoCols, label=True)
plt.show()
f.savefig('gradient_data/figures/yeo7.lat.png')
f = plot_surf_stat_map(surfL[0], surfL[1], bg_map=sulcL, mask=c, bg_on_stat=True, stat_map=y, azim=180, alpha=1, cmap=yeoCols, label=True)
plt.show()
f.savefig('gradient_data/figures/yeo7.med.png')

In [ ]:
# plot order on surface
y = yeo7L.copy()
ord1 = np.argsort(ord)
for n, v in enumerate(uniqueY):
    y[yeo7L == v] = int(ord1[n]) + 1 - 3
y[yeo7L == 37] = - 4
c = np.where([yeo17L != 37])[1]
f = plot_surf_stat_map(surfL[0], surfL[1], bg_map=sulcL, stat_map=y, mask=c, bg_on_stat=True, azim=0, alpha=1, label=True)
plt.show()
f.savefig('gradient_data/figures/yeo7.reord.lat.png')
f = plot_surf_stat_map(surfL[0], surfL[1], bg_map=sulcL, stat_map=y, mask=c, bg_on_stat=True, azim=180, alpha=1, label=True)
plt.show()
f.savefig('gradient_data/figures/yeo7.reord.med.png')

## Yeo17 networks

# Mapping to NeuroSynth

In [ ]:
from neurosynth.base.dataset import Dataset
from neurosynth.analysis import decode
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
# Import neurosynth database:
pickled_dataset = 'gradient_data/neurosynth/dataset.pkl'
dataset = Dataset.load(pickled_dataset)

# Import topics:
features = pd.read_csv('gradient_data/neurosynth/v3-topics-50.txt', sep='\t', index_col=0)
topics_to_keep = [4, 6, 14, 18, #17, 
                  23, 25, 20, #21,#27,
                  29, 30, 31, 33, 35, 
                  36,  38,  45, 37,
                  48, 49]
labels = ['semantics', 'cued attention', 'working memory',  'autobiographical memory', #'reading',
          'inhibition', 'motor', 'visual perception', #'number',#'reward',
          'visual attention', 'multisensory perception', 'visuospatial','eye movements', 'action',
          'auditory perception',  'pain',  'emotion',  'language', 
          'control', 'social cognition']            
features = features.iloc[:, topics_to_keep]
features.columns = labels
dataset.add_features(features, append=False)

In [ ]:
# include code to convert surface to volume...

In [ ]:
% matplotlib inline

decoder = decode.Decoder(dataset, method='roi')

tot = 5
data = decoder.decode([str('gradient_data/masks/volume_%02d_%02d.nii.gz' % (i * tot, (i * tot) + tot)) for i in xrange(0,100/tot)])

df = []
df = data.copy()
newnames = []
[newnames.append(('%s-%s' % (str(i * tot), str((i*tot) + tot)))) for i in xrange(0,len(df.columns))]
df.columns = newnames

# Set threshold:
thr = 3.1
vmin = 0
vmax = 15

def getOrder(d, thr):
    dh = []
    for i in range(0,len(d)):
        di = d[i]
        dh.append(np.average(np.array(xrange(0,len(d[i]))) + 1, weights=di))
    heatmapOrder = np.argsort(dh)
    return heatmapOrder

df[df<thr] = 0 
heatmapOrder = getOrder(np.array(df), thr)

sns.set(context="paper", font="sans-serif", font_scale=2)
f, (ax1) = plt.subplots(nrows=1,ncols=1,figsize=(30, 10), sharey=True)
plotData = df.reindex(df.index[heatmapOrder])
cbar_kws = { 'ticks' : [ thr, vmax ] }

cax = sns.heatmap(plotData, linewidths=1, square=True, cmap='Greys', robust=False, 
            ax=ax1, vmin=thr, vmax=vmax, mask=plotData == 0) # center = 0, cbar_kws=cbar_kws
sns.axlabel('Percentile along gradient', 'NeuroSynth topics terms')

# Supplementary Materials